# Wine quality assessment with Random Forest

In this tutorial I would like to give a brief overview of the functionality of scikit-learn and the Random Forest Regressor. First, we will download a dataset containing information about different kinds of wines. Then we will use packages of scikit-learn to preprocess the data and to predict the quality of an unseen wine bottle.

## Table of Conduct
[Section 1 - Import of Libraries and Modules](#section1)<br>
[Section 2 - Download of the dataset](#section2)<br>
[Section 3 - Split data in train and test set](#section3)<br>
[Section 4 - Preprocessing](#section4)<br>
[Section 5 - Cross Validation](#section5)<br>
[Section 6 - Testing the model](#section6)<br>
[Section 7 - Store the model](#section7)

<a id='section1'></a>
## Section 1 - Import of Libraries and Modules

In [89]:
import numpy as np
import pandas as pd

First, we will import some modules that we use later for splitting our data in train and test set and for getting the data in the correct representation for our regressor.

In [90]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

As said before, I will use Random Forest as Regressor in this tutorial. I assume that you have some basic knowledge about Tree Classifier and Random Forrest and will only cover the python view on that topic. For more information about the functionality of that regressor please read this: <a href="https://en.wikipedia.org/wiki/Random_forest">Random Forest</a>. To use Random Forest simply import scikit-learn's Module

In [91]:
from sklearn.ensemble import RandomForestRegressor

Next we need another Module to perform Cross-Validation.

In [92]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

Then we need some tool to evaluate the performance of the classification.

In [8]:
from sklearn.metrics import mean_squared_error, r2_score

To store the trained model for future use, we need one last module from sklearn. There is an alternative from python called pickle, but scikit's joblib works more efficient on large numpy arrays.

In [9]:
from sklearn.externals import joblib

<a id='section2'></a>
## Section 2 - Download the Dataset

Pandas provides some very helpful functions for importing data sets. One of these functions is read_csv(). With the help of these function data can be read from a CSV file without any problems. If, as in our case, the value names are in the first line, pandas will automatically use them to name the columns in the data object 

In [18]:
data = pd.read_csv(
    'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',
    sep=';'
)

In the upper cell, the data was downloaded in CSV format from the specified source and stored in a DataFrame object. In order to get an overview of the data set, the header of the data can be output in structured form as follows.

In [21]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


To get more information about the dimensionality of the data we can use:

In [23]:
data.shape

(1599, 12)

This shows us that the DataFrame consists of 1599 rows and 12 columns.
The describe function shows some statistics of the values that are stored in the dataset.

In [27]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


<a id="section3"></a>
## Section 3 - Split data in train and test set

To check how precisely our regressor works we first have to divide our data into a training and a test set. The training set is used to train the model, in other words to build the Decision Trees. The data from the training set is then applied to the trained model and the result is compared with the actual result.

In [83]:
y = data['quality']
X = data.drop('quality',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

A good rule of thumb is that you should divide the data about two thirds to one third into training and test data. The random_state parameter sets the seed value of the random division and stratified is set to yes because this way the distribution of the different labels in the trainings and testset is the same. 

<a id="section4"></a>
## Section 4 - Preprocessing

If we take a closer look at the data, we notice that the individual values differ greatly from one another. Many machine learning algorithms assume that the data is standardized, i.e. the expected value of the underlying distribution is 0 and the standard deviation is 1.

In [84]:
X_train_scaled = preprocessing.scale(X_train)
X_train_scaled

array([[-0.84424667,  0.58624775, -0.5362895 , ...,  0.60746272,
        -0.41407965, -0.11550523],
       [ 0.27720874,  0.39293761,  0.08152985, ...,  0.34229354,
        -0.97537481, -1.24081658],
       [-0.60815079, -0.43553443,  0.44192447, ...,  0.14341666,
         0.93302874,  0.07204666],
       ...,
       [-1.13936651,  0.2272432 , -1.30856368, ...,  1.00521648,
         0.09108599, -0.8657128 ],
       [ 0.9264724 ,  0.06154879,  0.90528898, ...,  0.54117042,
        -0.2456911 , -0.95948874],
       [-0.0179111 , -1.20877499,  0.59637931, ...,  0.01083207,
         0.59625164,  1.94756558]])

Now we have the following problem. The training data has been standardized, but the test data has not yet been standardized. If we were to apply the function to the test data now, the transformation would not take place with the same expected values and standard deviations. 
To solve this problem we use the <b>Standard Scaler</b>. We pass the training data to the Standard Scaler so that it can calculate the parameters necessary for standardization. Next we can use the Standard Scaler to standardize both the training and the test data with the same parameters. 

In [85]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Now both data sets were<b> standardized</b> with the same expected value and the same standard deviation. Since we want to use a cross validation pipeline later, we can also pass the standard scaler and regressor to it. The data is then automatically standardized in each step.

In [93]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                        RandomForestRegressor(n_estimators=100))

<a id="section5"></a>
## Section 5 - Cross Validation

Several techniques exist to evaluate the quality of the model. One of them is called<b> Cross Validation</b>. Here the data set is divided into even parts (usually 10). The model is trained with nine of the ten parts and tested with the tenth part. Here we can use the Cross Validation to further split the training set to find the optimal hyper parameters. The optimal model will then be checked again with the unseen test data. 

In [104]:
hyperparameters = { 
    'randomforestregressor__criterion': ['mse', 'mae'],
    'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
    'randomforestregressor__max_depth': [None, 5, 3, 1],
}

A list of all available parameters can be output as follows. 

In [105]:
pipeline.get_params()

{'memory': None,
 'steps': [('standardscaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('randomforestregressor',
   RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                         max_features='auto', max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=100,
                         n_jobs=None, oob_score=False, random_state=None,
                         verbose=0, warm_start=False))],
 'verbose': False,
 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1,

In [108]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)

# Train the model
clf = clf.fit(X_train, y_train)

In the previous cell we have written all the code necessary to perform a grid search, i.e. to test all specified parameter combinations with k-Fold Cross validation. 

In [109]:
clf.best_params_

{'randomforestregressor__criterion': 'mse',
 'randomforestregressor__max_depth': None,
 'randomforestregressor__max_features': 'log2'}

The grid search has shown that the most suitable metric is<b> MSE</b>. The maximum depth should be set to<b> None</b> and the maximum number of features to <b>log2</b>. The model has already been automatically trained with these values. 

In [110]:
clf.refit

True

<a id='section6'></a>
## Section 6 - Testing the model

Now let's check how well our model performs on the unseen test data. 

In [111]:
y_pred = clf.predict(X_test)

The quality of the wines of the training set were now predicted and stored in y_pred. Now we check how high the match with the real values is. 

In [113]:
r2_score(y_test, y_pred)

0.45809748309125176

In [114]:
mean_squared_error(y_test, y_pred)

0.35062405303030303

<a id='section7'></a>
## Section 7 - Store the model

After successfully training the model and achieving acceptable values, we can now save it and apply it to new data in the future. 

In [117]:
joblib.dump(clf, 'saved_models/rf_regressor.pkl')

['saved_models/rf_regressor.pkl']

To reload the model, just execute the following code: 

In [120]:
clf_reload = joblib.load('saved_models/rf_regressor.pkl')
y_pred_reload = clf_reload.predict(X_test)